## 보팅방식 모델 구현<hr>
- 데이터 : sklearn.datasets의 breast_cancer
- 유 형 : 지도학습 + 분류
- 방 법 : Voting 방식으로 진행 => LogisticRegression, DecisionTree, SVC
- 학습데이터셋 : 동일한 데이터셋으로 3개의 모델로 학습 진행

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
x,y = load_breast_cancer(as_frame=True, return_X_y=True)
x.shape, y.shape

((569, 30), (569,))

In [3]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,
                                             test_size=0.2,
                                             random_state=5,
                                             stratify=y)

[2] 학습 진행 <hr>

[2-1] 앙상블 보팅 학습에 사용할 모델 인스턴스 생성

In [ ]:
# LogisticRegression 인스턴스
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver='liblinear')
lr_model.fit(xtrain,ytrain)

In [0]:
# DecisionTreeClassifier 인스턴스
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=5)
dt_model.fit(xtrain,ytrain)

In [20]:
# SVC 인스턴스
from sklearn.svm import SVC

svc_model = SVC(random_state=5, probability=True) # probability=True -> predict_proba 사용 가능
svc_model.fit(xtrain, ytrain)

SVC(random_state=5)

[2-2] Ensomble 알고리즘 기반 분류

In [21]:
from sklearn.ensemble import VotingClassifier

In [26]:
# 동일 데이터셋으로 병렬 학습 진행할 모델 리스트 선정 및 결과 결정 방법 설정
vt_models = VotingClassifier(estimators=[('lr_model',lr_model),  # 이름은 마음대로
                                            ('dt_model',dt_model),
                                            ('svc_model',svc_model)],
                                verbose=True) # voting = 'soft' -> prodict_proba 사용 가능

vt_models.fit(xtrain,ytrain)

[Voting] ................. (1 of 3) Processing lr_model, total=   0.0s
[Voting] ................. (2 of 3) Processing dt_model, total=   0.0s
[Voting] ................ (3 of 3) Processing svc_model, total=   0.0s


VotingClassifier(estimators=[('lr_model',
                              LogisticRegression(solver='liblinear')),
                             ('dt_model',
                              DecisionTreeClassifier(random_state=5)),
                             ('svc_model', SVC(random_state=5))],
                 verbose=True)

In [45]:
# 예측하기 
newdata =pd.DataFrame([xtest.iloc[0]], columns=xtest.columns)
# newdata = xtest.iloc[0].to_frame().T  ->  이렇게도 가능

vt_models.predict(newdata)

array([1])

In [ ]:
# 보팅 인스턴스 내의 학습기들 => 접근 방법(1) - 인덱싱으로 접근
vt_models.estimators_

In [42]:
# 보팅 인스턴스 내의 학습기들 => 접근 방법(2) - key로 접근
# named_estimators_ : dict형으로 존재
vt_models.named_estimators_.get('lr_model').predict(newdata),vt_models.named_estimators_.get('dt_model').predict(newdata),vt_models.named_estimators_.get('svc_model').predict(newdata)

(array([1], dtype=int64), array([1], dtype=int64), array([1], dtype=int64))

In [44]:
for key, value in vt_models.named_estimators_.items():
    print(key, value.predict(newdata)[0])

lr_model 1
dt_model 1
svc_model 1


- 3개 알고리즘 모델에 대한 성능 평가

In [19]:
print(f'[Logistic Regression] Train score: {lr_model.score(xtrain,ytrain)} Test score: {lr_model.score(xtest,ytest)}')
print(f'[DecisionTreeClassifier] Train score: {dt_model.score(xtrain, ytrain)} Test score: {dt_model.score(xtest,ytest)}') # 과대적합
print(f'[RandomForestClassifier] Train score: {svc_model.score(xtrain,ytrain)} Test score: {svc_model.score(xtest,ytest)}')  # 과대적합

[Logistic Regression] Train score: 0.9582417582417583 Test score: 0.9649122807017544
[DecisionTreeClassifier] Train score: 1.0 Test score: 0.9210526315789473
[RandomForestClassifier] Train score: 0.9164835164835164 Test score: 0.8771929824561403


In [ ]:
rf_model.feature_importances_